In [1]:
from openvino.inference_engine import IENetwork, IEPlugin
import cv2

In [2]:
detection_model_path = '/media/prateek/prateek_space/intel_model_dump/intel/vehicle-detection-adas-0002/FP32/vehicle-detection-adas-0002.xml'
attr_model_path = '/media/prateek/prateek_space/intel_model_dump/intel/vehicle-attributes-recognition-barrier-0039/FP32/vehicle-attributes-recognition-barrier-0039.xml'
cpu_extension = '../build_samples/intel64/Release/lib/libcpu_extension.so'
device = 'CPU'
max_batch_size = 20

In [11]:
net = IENetwork(model=detection_model_path, weights=detection_model_path.replace("xml","bin"))
input_blob = next(iter(net.inputs))
out_blob = list(net.outputs.keys())
plugin = IEPlugin(device="CPU")
plugin.add_cpu_extension(cpu_extension)
# Set max batch size
net.batch_size = 30
batch_size, channel, height, width = net.inputs[input_blob].shape

# plugin.set_config({"DYN_BATCH_ENABLED": "YES"})
exec_net = plugin.load(network=net)
# Set batch size for certain network.


In [4]:
def preprocess_frame(frame):
    """
    after processing cursor id is updated
    """
    in_frame = cv2.resize(frame.copy(), (width, height))
    in_frame = in_frame.transpose((2, 0, 1))  # Change data layout from HWC to CHW
    in_frame = in_frame.reshape((1, channel, height, width))
    return in_frame

In [5]:
im = cv2.imread("/media/prateek/shared_space/Notebooks/abg/production_code/vehicle_detection/sample_image.jpeg")

In [7]:
im1 = preprocess_frame(im)

In [14]:
import numpy as np

In [20]:
%%timeit
exec_net.infer({input_blob:[im1]})

301 ms ± 3.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
exec_net.infer({input_blob:[im1]*30})

301 ms ± 2.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
from openvino.inference_engine import IENetwork, IEPlugin
import cv2
class dynamic_batch_infer()